# Waste Detection with YOLOv8 and FastAPI

This notebook sets up and runs a waste detection system using YOLOv8 for training and inference, and FastAPI for serving the model. It includes steps to download the dataset from Roboflow, train the model, run the FastAPI app with `best.pt`, and optionally convert the model to RKNN format for deployment on OrangePi5Pro.

**Dataset:**
- The dataset is sourced from Roboflow Universe: [Waste Detection Dataset](https://universe.roboflow.com/ai-project-i3wje/waste-detection-vqkjo).
- It contains 9178 images of recyclable and non-recyclable waste, annotated for object detection with 22 classes.

**Requirements:**
- Python 3.8-3.11 with dependencies listed in `requirements.txt`.
- A Roboflow API key to download the dataset (sign up at [Roboflow](https://roboflow.com) to get one).
- GPU (optional but recommended for training).
- Webcam (for testing FastAPI video feed).

**Steps:**
1. Download and set up the dataset from Roboflow.
2. Train the YOLOv8 model.
3. Run the FastAPI app with `best.pt`.
4. (Optional) Convert `best.pt` to RKNN and run FastAPI with RKNN model.

**Dataset Setup Tutorial:**
1. **Obtain Roboflow API Key:**
   - Sign up or log in at [Roboflow](https://roboflow.com).
   - Navigate to your account settings to find your API key.
   - Provide this key when prompted in the dataset download cell below.

2. **Download the Dataset:**
   - The notebook will download the dataset using the Roboflow API to the `dataset` directory.
   - The dataset will be in YOLO format, with `train`, `valid`, and `test` folders, each containing `images` and `labels` subfolders.
   - Alternatively, you can manually download the dataset from [https://universe.roboflow.com/ai-project-i3wje/waste-detection-vqkjo](https://universe.roboflow.com/ai-project-i3wje/waste-detection-vqkjo) in YOLOv8 format and extract it to the `dataset` directory.

3. **Place the Dataset Folders:**
   - After downloading, the dataset will be automatically placed in the `dataset` directory with the following structure:
     ```
     dataset/
     ├── train/
     │   ├── images/
     │   │   ├── image1.jpg
     │   │   ├── image2.jpg
     │   │   └── ...
     │   └── labels/
     │       ├── image1.txt
     │       ├── image2.txt
     │       └── ...
     ├── valid/
     │   ├── images/
     │   └── labels/
     ├── test/
     │   ├── images/
     │   └── labels/
     └── data.yaml
     ```
   - Ensure the `data.yaml` file is in the `dataset` directory, as it specifies the paths to `train`, `valid`, and `test` images and the 22 class names.
   - Do not modify the folder structure or `data.yaml` unless you know what you're doing, as the training script relies on this setup.

4. **Verify the Dataset:**
   - The notebook includes a verification step to check that the `train`, `valid`, and `test` directories contain images.
   - If you encounter errors, ensure the dataset was downloaded correctly and the folders are populated.

Run all cells sequentially to execute the pipeline. If you already have a trained `best.pt`, place it in the `weights` directory and skip the training step.

In [ ]:
# Install Roboflow library if not already installed
# --quiet flag suppresses verbose output for cleaner notebook
!pip install ultralytics --quiet
!pip install torch --quiet
!pip install onnxruntime --quiet
!pip install roboflow --quiet

# Import necessary libraries for file handling
import os
from pathlib import Path
from roboflow import Roboflow

# Define base directory: current working directory of the notebook
BASE_DIR = Path.cwd()

# Create necessary project directories
print(f"Creating project directories under: {BASE_DIR}")
os.makedirs(BASE_DIR / 'dataset', exist_ok=True)
os.makedirs(BASE_DIR / 'weights', exist_ok=True)
os.makedirs(BASE_DIR / 'static', exist_ok=True)
print("Project directories created: dataset, weights, static")

# Change directory to where the dataset will be downloaded.
# Roboflow will create a subfolder (e.g., 'waste-detection-9') inside this.
print(f"Changing current directory to: {BASE_DIR / 'dataset'}")
%cd {BASE_DIR}/dataset

# Download dataset from Roboflow
# NOTE: Roboflow automatically generates a data.yaml file
# inside the downloaded dataset folder. You typically use that.
print("Starting dataset download from Roboflow...")
rf = Roboflow(api_key="HI42QRXkU9xSlq7DHhks")
project = rf.workspace("ai-project-i3wje").project("waste-detection-vqkjo")
dataset = project.version(9).download("yolov8") # This creates a 'waste-detection-9' folder (or similar)

# Get the actual path where Roboflow downloaded the dataset.
# The `dataset.location` attribute gives the path to the main dataset folder (e.g., 'BASE_DIR/dataset/waste-detection-9')
ROBOFLOW_DATASET_ROOT = Path(dataset.location)
print(f"Roboflow dataset downloaded to: {ROBOFLOW_DATASET_ROOT}")

# --- Creating/Verifying data.yaml consistency ---
# Define the content of your data.yaml file.
# These paths are relative to the ROBOFLOW_DATASET_ROOT (e.g., 'waste-detection-9/')
# This ensures consistency with the Roboflow's internal structure.
data_yaml_content = f"""
train: {ROBOFLOW_DATASET_ROOT.name}/train/images
val: {ROBOFLOW_DATASET_ROOT.name}/valid/images
test: {ROBOFLOW_DATASET_ROOT.name}/test/images

nc: 22
names: ['battery', 'can', 'cardboard_bowl', 'cardboard_box', 'chemical_plastic_bottle', 'chemical_plastic_gallon', 'chemical_spray_can', 'light_bulb', 'paint_bucket', 'plastic_bag', 'plastic_bottle', 'plastic_bottle_cap', 'plastic_box', 'plastic_cultery', 'plastic_cup', 'plastic_cup_lid', 'reuseable_paper', 'scrap_paper', 'scrap_plastic', 'snack_bag', 'stick', 'straw']
"""

# Define the full path for the data.yaml file.
# It's usually placed directly inside the main dataset folder created by Roboflow.
data_yaml_path = ROBOFLOW_DATASET_ROOT / 'data.yaml'

# Write the content to the data.yaml file, overwriting if it exists
# (Roboflow already creates one, but we ensure its content matches if needed)
try:
    with open(data_yaml_path, 'w') as f:
        f.write(data_yaml_content.strip())
    print(f"\ndata.yaml successfully created/updated at: {data_yaml_path}")
except Exception as e:
    print(f"Error writing data.yaml: {e}")

# Define dataset paths based on the actual Roboflow download structure
# Note: DATASET_DIR here is the BASE_DIR / 'dataset'
# The actual data is inside ROBOFLOW_DATASET_ROOT (e.g., BASE_DIR / 'dataset' / 'waste-detection-9')
DATASET_DIR = ROBOFLOW_DATASET_ROOT # This now points to the correct root for the dataset
TRAIN_IMAGES = DATASET_DIR / 'train' / 'images'
VAL_IMAGES = DATASET_DIR / 'valid' / 'images'
TEST_IMAGES = DATASET_DIR / 'test' / 'images'

print('\nDataset paths defined:')
print(f'DATASET_DIR: {DATASET_DIR}')
print(f'TRAIN_IMAGES: {TRAIN_IMAGES}')
print(f'VAL_IMAGES: {VAL_IMAGES}')
print(f'TEST_IMAGES: {TEST_IMAGES}')
# --- Verification of data.yaml existence and content ---
if not data_yaml_path.exists():
    raise FileNotFoundError(f'data.yaml not found at {data_yaml_path}. Ensure dataset downloaded correctly or path is wrong.')

# Read and verify data.yaml content
print('\nVerifying data.yaml content:')
try:
    with open(data_yaml_path, 'r') as f:
        data_yaml_content_read = f.read()
    print(data_yaml_content_read)
    print("data.yaml content verified.")
except Exception as e:
    print(f"Error reading data.yaml for verification: {e}")

print('\nDataset downloaded and directory structure verified.')
print('Ready to proceed with training or further steps!')


In [ ]:
# Import necessary libraries (ensure they are imported if this cell is run independently)
import shutil
from pathlib import Path

# Assume BASE_DIR and ROBOFLOW_DATASET_ROOT are defined from previous cells
# If running this cell standalone, uncomment and define them:
# BASE_DIR = Path.cwd()
# ROBOFLOW_DATASET_ROOT = BASE_DIR / 'dataset' / 'waste-detection-9' # Adjust 'waste-detection-9' to your actual folder name

# --- Create train.py and run it ---
# train.py content
train_py_content = f"""from ultralytics import YOLO
import torch
from pathlib import Path
import sys # Added this import to handle __spec__ issue

# Absolute path to the Roboflow dataset root, passed from the notebook
ROBOFLOW_DATASET_ROOT_ABS_PATH = r"{ROBOFLOW_DATASET_ROOT.as_posix()}"

def main():
    # Define path to data.yaml using the absolute path
    data_yaml = str(Path(ROBOFLOW_DATASET_ROOT_ABS_PATH) / 'data.yaml')

    # Load YOLOv8n model (pre-trained)
    model = YOLO('yolov8n.pt')

    # Training
    print('Starting training...')
    train_results = model.train(
        data=data_yaml,
        epochs=50,
        imgsz=640,
        batch=16,
        device=0 if torch.cuda.is_available() else 'cpu',
        name='waste_detection'
    )

    # Validation
    print('Starting validation...')
    val_results = model.val(
        data=data_yaml,
        imgsz=640,
        batch=16,
        device=0 if torch.cuda.is_available() else 'cpu'
    )

    # Prediction on test data
    print('Starting prediction...')
    predict_results = model.predict(
        source=str(Path(ROBOFLOW_DATASET_ROOT_ABS_PATH) / 'test' / 'images'),
        save=True,
        imgsz=640,
        device=0 if torch.cuda.is_available() else 'cpu'
    )

    print('Process completed!')
    print(f'Training results saved at: runs/train/waste_detection')
    print(f'Best model: runs/train/waste_detection/weights/best.pt')
    print(f'Prediction results saved at: runs/predict/')

if __name__ == '__main__':
    # Workaround for AttributeError: module '__main__' has no attribute '__spec__'
    # when running with %run in some Jupyter/IPython environments.
    if not hasattr(sys.modules['__main__'], '__spec__'):
        class DummySpec:
            name = '__main__'
        sys.modules['__main__'].__spec__ = DummySpec()
    main()
"""

# Write train.py to the BASE_DIR
with open(BASE_DIR / 'train.py', 'w') as f:
    f.write(train_py_content)

print('train.py created successfully at:', BASE_DIR / 'train.py')

# Change directory back to BASE_DIR before running train.py
print(f"Changing current directory back to: {BASE_DIR}")
%cd {BASE_DIR}

# Run training
# Note: This step may take significant time depending on your hardware and dataset size.
# If you already have a trained 'best.pt', you can skip this cell and copy 'best.pt' to the 'weights' directory.
print("Running train.py...")
%run train.py

# Move best.pt to weights directory
best_pt_source = BASE_DIR / 'runs' / 'train' / 'waste_detection' / 'weights' / 'best.pt'
best_pt_dest = BASE_DIR / 'weights' / 'best.pt'

if best_pt_source.exists():
    shutil.move(best_pt_source, best_pt_dest)
    print('best.pt moved to weights directory.')
else:
    # Check common reasons for failure
    if not (BASE_DIR / 'runs' / 'train' / 'waste_detection').exists():
        print("Error: 'runs/train/waste_detection' directory not found. Training might have failed or been interrupted.")
    else:
        print("Error: 'best.pt' not found in 'runs/train/waste_detection/weights/'. Check training logs for details.")
    raise FileNotFoundError('Training did not produce best.pt. Check training logs for errors.')


In [ ]:
# Create settings.py
settings_py_content = """from pathlib import Path
import sys

file_path = Path(__file__).resolve()
root_path = file_path.parent
if root_path not in sys.path:
    sys.path.append(str(root_path))
ROOT = root_path.relative_to(Path.cwd())

# ML Model config
MODEL_DIR = ROOT / 'weights'
DETECTION_MODEL = MODEL_DIR / 'best.pt'

MODEL_INPUT_WIDTH = 640
MODEL_INPUT_HEIGHT = 640
CONF_THRESHOLD = 0.25
NMS_IOU_THRESHOLD = 0.45

# Class names
ALL_CLASSES = [
    'battery', 'can', 'cardboard_bowl', 'cardboard_box', 'chemical_plastic_bottle',
    'chemical_plastic_gallon', 'chemical_spray_can', 'light_bulb', 'paint_bucket',
    'plastic_bag', 'plastic_bottle', 'plastic_bottle_cap', 'plastic_box',
    'plastic_cultery', 'plastic_cup', 'plastic_cup_lid', 'reuseable_paper',
    'scrap_paper', 'scrap_plastic', 'snack_bag', 'stick', 'straw'
]

# Webcam
WEBCAM_PATH = 0

# Waste type classification
RECYCLABLE = ['cardboard_box', 'can', 'plastic_bottle_cap', 'plastic_bottle', 'reuseable_paper']
NON_RECYCLABLE = [
    'plastic_bag', 'scrap_paper', 'stick', 'plastic_cup', 'snack_bag',
    'plastic_box', 'straw', 'plastic_cup_lid', 'scrap_plastic',
    'cardboard_bowl', 'plastic_cultery'
]
HAZARDOUS = [
    'battery', 'chemical_spray_can', 'chemical_plastic_bottle',
    'chemical_plastic_gallon', 'light_bulb', 'paint_bucket'
]
"""

with open(BASE_DIR / 'settings.py', 'w') as f:
    f.write(settings_py_content)

print('settings.py created successfully.')

In [ ]:
# Import necessary libraries
from pathlib import Path

# Assume BASE_DIR is already defined from previous cells or define it here if running standalone
BASE_DIR = Path.cwd() # Adjust if your BASE_DIR is different

# script.js content
script_js_content = """document.addEventListener('DOMContentLoaded', () => {
    const dropArea = document.getElementById('drop-area');
    const fileInput = document.getElementById('file-input');
    const uploadBtn = document.getElementById('upload-btn');
    const imagePreview = document.getElementById('image-preview');
    const previewImage = imagePreview.querySelector('img');
    const uploadStatus = document.getElementById('upload-status');
    const resultImage = document.getElementById('result-image');
    const resultText = document.getElementById('result-text');

    const startWebcamBtn = document.getElementById('start-webcam');
    const stopWebcamBtn = document.getElementById('stop-webcam');
    const webcam = document.getElementById('webcam');
    const webcamResult = document.getElementById('webcam-result');
    const webcamStatus = document.getElementById('webcam-status');

    let pollClassificationInterval = null;

    const showMessage = (element, message, type = 'info') => {
        element.classList.remove('hidden', 'text-green-500', 'text-red-500', 'text-blue-500', 'bg-green-100', 'bg-red-100', 'bg-blue-100');
        element.classList.add('p-2', 'rounded');
        if (type === 'success') {
            element.classList.add('bg-green-100', 'text-green-500');
        } else if (type === 'error') {
            element.classList.add('bg-red-100', 'text-red-500');
        } else if (type === 'info') {
            element.classList.add('bg-blue-100', 'text-blue-500');
        }
        element.innerHTML = message;
    };

    const hideMessage = (element) => {
        element.classList.add('hidden');
        element.innerHTML = '';
    };

    // --- Image Upload Functionality ---
    dropArea.addEventListener('dragover', (e) => {
        e.preventDefault();
        dropArea.classList.add('border-blue-500', 'bg-blue-50');
    });

    dropArea.addEventListener('dragleave', () => {
        dropArea.classList.remove('border-blue-500', 'bg-blue-50');
    });

    dropArea.addEventListener('drop', (e) => {
        e.preventDefault();
        dropArea.classList.remove('border-blue-500', 'bg-blue-50');
        const file = e.dataTransfer.files[0];
        if (file && file.type.startsWith('image/')) {
            fileInput.files = e.dataTransfer.files;
            displayImagePreview(file);
        } else {
            showMessage(uploadStatus, 'Please drop an image file.', 'error');
        }
    });

    dropArea.addEventListener('click', () => {
        fileInput.click();
    });

    fileInput.addEventListener('change', () => {
        if (fileInput.files.length > 0) {
            displayImagePreview(fileInput.files[0]);
        } else {
            imagePreview.classList.add('hidden');
            previewImage.src = '#';
        }
    });

    function displayImagePreview(file) {
        const reader = new FileReader();
        reader.onload = (e) => {
            previewImage.src = e.target.result;
            imagePreview.classList.remove('hidden');
        };
        reader.readAsDataURL(file);
    }

    uploadBtn.addEventListener('click', async () => {
        if (fileInput.files.length === 0) {
            showMessage(uploadStatus, 'Please select an image to upload!', 'error');
            return;
        }

        hideMessage(uploadStatus);
        resultImage.innerHTML = '';
        resultText.innerHTML = '';
        showMessage(uploadStatus, 'Uploading and detecting...', 'info');

        const formData = new FormData();
        formData.append('file', fileInput.files[0]);

        try {
            const response = await fetch('/detect', {
                method: 'POST',
                body: formData
            });

            if (!response.ok) {
                const errorData = await response.json();
                throw new Error(errorData.detail || 'Something went wrong during detection.');
            }

            const data = await response.json();
            showMessage(uploadStatus, 'Upload and detection successful!', 'success');

            resultImage.innerHTML = `<img src="data:image/jpeg;base64,${data.image}" alt="Detected Image" class="w-full rounded">`;

            let resultHtml = '';
            if (data.results.recyclable.length > 0) {
                resultHtml += `<div class="bg-yellow-200 p-4 rounded"><strong>Recyclable items:</strong><br>- ${data.results.recyclable.join('<br>- ')}</div>`;
            }
            if (data.results.non_recyclable.length > 0) {
                resultHtml += `<div class="bg-blue-200 p-4 rounded"><strong>Non-Recyclable items:</strong><br>- ${data.results.non_recyclable.join('<br>- ')}</div>`;
            }
            if (data.results.hazardous.length > 0) {
                resultHtml += `<div class="bg-red-200 p-4 rounded"><strong>Hazardous items:</strong><br>- ${data.results.hazardous.join('<br>- ')}</div>`;
            }
            resultText.innerHTML = resultHtml;

        } catch (error) {
            showMessage(uploadStatus, `Error: ${error.message}`, 'error');
            resultImage.innerHTML = '';
            resultText.innerHTML = '';
        }
    });

    // --- Webcam Functionality ---
    startWebcamBtn.addEventListener('click', async () => {
        // Clear previous states
        hideMessage(webcamStatus);
        webcamResult.innerHTML = '';

        showMessage(webcamStatus, 'Starting webcam...', 'info');

        // Reset webcam src to force reload the stream
        webcam.src = ''; 
        webcam.src = '/video_feed';
        webcam.classList.remove('hidden');
        startWebcamBtn.classList.add('hidden');
        stopWebcamBtn.classList.remove('hidden');

        // Set up onload and onerror for webcam image
        webcam.onload = () => {
            showMessage(webcamStatus, 'Webcam stream started.', 'success');
        };
        webcam.onerror = () => {
            showMessage(webcamStatus, 'Error loading webcam stream. Check backend logs or webcam settings.', 'error');
            webcam.classList.add('hidden');
            startWebcamBtn.classList.remove('hidden');
            stopWebcamBtn.classList.add('hidden');
        };

        // Start polling for classification data
        if (pollClassificationInterval) {
            clearInterval(pollClassificationInterval);
        }
        pollClassificationInterval = setInterval(async () => {
            try {
                const response = await fetch('/webcam_classification');
                if (!response.ok) {
                    throw new Error('Failed to fetch classification data.');
                }
                const data = await response.json();

                let resultHtml = '';
                if (data.recyclable.length > 0) {
                    resultHtml += `<div class="bg-yellow-200 p-4 rounded"><strong>Recyclable items:</strong><br>- ${data.recyclable.join('<br>- ')}</div>`;
                }
                if (data.non_recyclable.length > 0) {
                    resultHtml += `<div class="bg-blue-200 p-4 rounded"><strong>Non-Recyclable items:</strong><br>- ${data.non_recyclable.join('<br>- ')}</div>`;
                }
                if (data.hazardous.length > 0) {
                    resultHtml += `<div class="bg-red-200 p-4 rounded"><strong>Hazardous items:</strong><br>- ${data.hazardous.join('<br>- ')}</div>`;
                }
                webcamResult.innerHTML = resultHtml;

            } catch (error) {
                console.error('Frontend: Error fetching webcam classification:', error);
                // Only show error if the webcam stream itself isn't stopped
                if (webcam.src !== '') {
                    // Avoid spamming error messages if backend is just shutting down
                    if (!error.message.includes("Failed to fetch")) { // Simple check to avoid network errors during shutdown
                        showMessage(webcamStatus, `Error fetching classification: ${error.message}`, 'error');
                    }
                }
            }
        }, 1000); // Poll every 1 second (adjust as needed)
    });

    stopWebcamBtn.addEventListener('click', async () => {
        // First, tell the backend to stop the stream
        showMessage(webcamStatus, 'Sending stop signal to backend...', 'info');
        try {
            const response = await fetch('/stop_webcam_backend', { 
                method: 'POST',
                // Optional: set headers if needed, e.g., 'Content-Type': 'application/json' for body
            });
            if (!response.ok) {
                // If backend returns an error status
                const errorData = await response.json();
                console.error('Frontend: Failed to send stop signal to backend:', errorData.message);
                showMessage(webcamStatus, `Failed to stop webcam on backend: ${errorData.message}`, 'error');
            } else {
                console.log('Frontend: Stop signal sent to backend successfully.');
                // Backend will handle releasing the camera now
            }
        } catch (error) {
            // Network error (e.g., backend crashed, unreachable)
            console.error('Frontend: Error sending stop signal:', error);
            showMessage(webcamStatus, `Error sending stop signal: ${error.message}`, 'error');
        } finally {
            // Regardless of backend signal success, stop frontend display and polling
            if (webcam.src !== '') { 
                webcam.src = ''; // Stop the video stream by clearing src
                webcam.classList.add('hidden');
                startWebcamBtn.classList.remove('hidden');
                stopWebcamBtn.classList.add('hidden');
                webcamResult.innerHTML = '';
            }
            if (pollClassificationInterval) {
                clearInterval(pollClassificationInterval); // Stop polling
                pollClassificationInterval = null;
            }
            // Give a moment for backend to actually release before showing 'stopped' status
            setTimeout(() => {
                showMessage(webcamStatus, 'Webcam stopped successfully.', 'success');
            }, 1000); // Waktu yang cukup untuk backend melepaskan kamera (misal 1 detik)
        }
    });
});
"""

# Write script.js to the static directory
with open(BASE_DIR / 'static' / 'script.js', 'w') as f:
    f.write(script_js_content)

print('script.js created successfully at:', BASE_DIR / 'static' / 'script.js')


In [ ]:
# Import necessary libraries (ensure they are imported if this cell is run independently)
from pathlib import Path

# Assume BASE_DIR is already defined from previous cells or define it here if running standalone
BASE_DIR = Path.cwd() # Adjust if your BASE_DIR is different

# Create main.py (FastAPI app for best.pt)
main_py_content = """from fastapi import FastAPI, File, UploadFile, HTTPException
from fastapi.responses import HTMLResponse, StreamingResponse, JSONResponse
from fastapi.staticfiles import StaticFiles
from ultralytics import YOLO
import cv2
import numpy as np
from pathlib import Path
import settings
import io
from PIL import Image
import base64
import time
import asyncio

app = FastAPI()

app.mount('/static', StaticFiles(directory='static'), name='static')

model_path = Path(settings.DETECTION_MODEL)
try:
    model = YOLO(model_path)
except Exception as ex:
    raise Exception(f'Unable to load model. Check the specified path: {model_path} - {ex}')

latest_webcam_results = {
    'recyclable': [],
    'non_recyclable': [],
    'hazardous': []
}

webcam_stop_event = asyncio.Event()

def classify_waste_type(detected_items):
    recyclable_items = set(detected_items) & set(settings.RECYCLABLE)
    non_recyclable_items = set(detected_items) & set(settings.NON_RECYCLABLE)
    hazardous_items = set(detected_items) & set(settings.HAZARDOUS)
    return recyclable_items, non_recyclable_items, hazardous_items

def remove_dash_from_class_name(class_name):
    return class_name.replace('_', ' ')

@app.get('/', response_class=HTMLResponse)
async def serve_index():
    with open('static/index.html', 'r') as f:
        return f.read()

@app.post('/detect')
async def detect_image(file: UploadFile = File(...)):
    try:
        contents = await file.read()
        image = Image.open(io.BytesIO(contents)).convert('RGB')
        image = np.array(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        results = model.predict(image, conf=0.6, verbose=False)
        names = model.names
        detected_items = set([names[int(c)] for c in results[0].boxes.cls])

        recyclable_items, non_recyclable_items, hazardous_items = classify_waste_type(detected_items)
        
        result_dict = {
            'recyclable': [remove_dash_from_class_name(item) for item in recyclable_items],
            'non_recyclable': [remove_dash_from_class_name(item) for item in non_recyclable_items],
            'hazardous': [remove_dash_from_class_name(item) for item in hazardous_items]
        }

        res_plotted = results[0].plot()
        _, buffer = cv2.imencode('.jpg', res_plotted)
        encoded_image = base64.b64encode(buffer).decode('utf-8')

        return {'results': result_dict, 'image': encoded_image}
    except Exception as e:
        print(f'Error during image detection: {e}')
        raise HTTPException(status_code=500, detail=str(e))

@app.get('/video_feed')
async def video_feed():
    global latest_webcam_results
    webcam_stop_event.clear()
    print('Backend: Webcam stream started, stop event cleared.')

    async def generate():
        cap = cv2.VideoCapture(settings.WEBCAM_PATH)
        if not cap.isOpened():
            print('Backend: Error: Could not open webcam.')
            # Corrected indentation for yield continuation lines
            yield (b'--frame\\r\\n'
                   b'Content-Type: image/jpeg\\r\\n\\r\\n' +
                   cv2.imencode('.jpg', np.zeros((480, 640, 3), dtype=np.uint8))[1].tobytes() + b'\\r\\n')
            return

        print('Backend: Webcam opened successfully. Streaming frames...')
        
        prev_frame_time = 0
        new_frame_time = 0

        try:
            while True:
                if webcam_stop_event.is_set():
                    print('Backend: Webcam stop event detected. Breaking loop.')
                    break

                success, frame = cap.read()
                if not success:
                    print('Backend: Error: Failed to read frame from webcam. Breaking loop.')
                    break
                
                new_frame_time = time.time()
                fps = 1 / (new_frame_time - prev_frame_time)
                prev_frame_time = new_frame_time
                fps_text = f'FPS: {int(fps)}'

                results = model.predict(frame, conf=0.6, verbose=False)
                names = model.names
                detected_items = set([names[int(c)] for c in results[0].boxes.cls])

                recyclable_items, non_recyclable_items, hazardous_items = classify_waste_type(detected_items)
                
                latest_webcam_results['recyclable'] = [remove_dash_from_class_name(item) for item in recyclable_items]
                latest_webcam_results['non_recyclable'] = [remove_dash_from_class_name(item) for item in non_recyclable_items]
                latest_webcam_results['hazardous'] = [remove_dash_from_class_name(item) for item in hazardous_items]

                res_plotted = results[0].plot()
                cv2.putText(res_plotted, fps_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

                _, buffer = cv2.imencode('.jpg', res_plotted)
                frame_bytes = buffer.tobytes()

                # Corrected indentation for yield continuation lines
                yield (b'--frame\\r\\n'
                       b'Content-Type: image/jpeg\\r\\n\\r\\n' + frame_bytes + b'\\r\\n')
                
                await asyncio.sleep(0.001)
        finally:
            if cap.isOpened():
                cap.release()
                print('Backend: Webcam released.')
            else:
                print('Backend: Webcam was not opened, nothing to release.')

    return StreamingResponse(generate(), media_type='multipart/x-mixed-replace;boundary=frame')

@app.post('/stop_webcam_backend')
async def stop_webcam_backend():
    webcam_stop_event.set()
    print('Backend: Received stop signal from frontend. Event set.')
    return JSONResponse(content={'message': 'Webcam stop signal sent.'})

@app.get('/webcam_classification')
async def get_webcam_classification():
    return JSONResponse(content=latest_webcam_results)
"""

with open(BASE_DIR / 'main.py', 'w') as f:
    f.write(main_py_content)

print('main.py created successfully at:', BASE_DIR / 'main.py')

# Create a basic index.html for the FastAPI app
index_html_content = """<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Waste Detection</title>
    <script src="https://cdn.tailwindcss.com"></script>
</head>
<body class="bg-gray-100 font-sans">
    <div class="container mx-auto p-4">
        <h1 class="text-3xl font-bold text-center mb-4">Intelligent Waste Segregation System</h1>
        <p class="text-center mb-6">Upload an image or use your device camera to detect waste types.</p>

        <div class="flex flex-col md:flex-row gap-6">
            <div class="flex-1 bg-white p-6 rounded-lg shadow-md">
                <h2 class="text-xl font-semibold mb-4">Upload Image</h2>
                <div id="drop-area" class="border-2 border-dashed border-gray-400 p-6 text-center cursor-pointer hover:border-blue-500 hover:bg-blue-50 transition">
                    <p class="text-gray-600">Drag and drop an image here or click to upload</p>
                    <input type="file" id="file-input" accept="image/*" class="hidden">
                </div>
                <div id="image-preview" class="mt-4 hidden">
                    <p class="text-sm text-gray-600 mb-2">Image Preview:</p>
                    <img src="#" alt="Image Preview" class="w-full rounded max-h-64 object-contain">
                </div>
                <button id="upload-btn" class="mt-4 w-full bg-blue-500 text-white py-2 px-4 rounded hover:bg-blue-600">Upload and Detect</button>
                <div id="upload-status" class="mt-4 text-center hidden"></div>
                <div id="result-image" class="mt-4"></div>
                <div id="result-text" class="mt-4 space-y-2"></div>
            </div>

            <div class="flex-1 bg-white p-6 rounded-lg shadow-md">
                <h2 class="text-xl font-semibold mb-4">Live Webcam Detection</h2>
                <div class="flex gap-2">
                    <button id="start-webcam" class="flex-1 bg-blue-500 text-white py-2 px-4 rounded hover:bg-blue-600">Start Webcam</button>
                    <button id="stop-webcam" class="flex-1 bg-red-500 text-white py-2 px-4 rounded hover:bg-red-600 hidden">Stop Webcam</button>
                </div>
                <div class="mt-4 flex flex-col md:flex-row gap-4">
                    <img id="webcam" src="" class="w-full md:w-2/3 rounded hidden">
                    <div id="webcam-result" class="w-full md:w-1/3 space-y-2"></div>
                </div>
                 <div id="webcam-status" class="mt-4 text-center hidden"></div>
            </div>
        </div>
    </div>

    <script src="/static/script.js"></script>
</body>
</html>
"""

with open(BASE_DIR / 'static' / 'index.html', 'w') as f:
    f.write(index_html_content)

print('index.html created successfully at:', BASE_DIR / 'static' / 'index.html')


In [ ]:
# Import necessary libraries
import subprocess
import time
from pathlib import Path
import os # Import os for path checking

# Assume BASE_DIR is already defined from previous cells or define it here if running standalone
BASE_DIR = Path.cwd() # Adjust if your BASE_DIR is different

# Ensure FastAPI and Uvicorn are installed
print("Ensuring FastAPI and Uvicorn are installed...")
!pip install fastapi --quiet
!pip install uvicorn --quiet
print("FastAPI and Uvicorn installation check complete.")

# Change directory to BASE_DIR before starting FastAPI
print(f"Changing current directory to: {BASE_DIR}")
%cd {BASE_DIR}

# --- Pre-check for best.pt model ---
# This is crucial as main.py will try to load it immediately
# Assuming settings.py defines DETECTION_MODEL correctly as BASE_DIR / 'weights' / 'best.pt'
model_path_check = BASE_DIR / 'weights' / 'best.pt'
if not model_path_check.exists():
    raise FileNotFoundError(
        f"Error: Model 'best.pt' not found at {model_path_check}. "
        "Please ensure the training cell completed successfully and 'best.pt' was moved to the 'weights' directory."
    )
print(f"Model '{model_path_check.name}' found. Proceeding to start FastAPI server.")

# Start FastAPI server
# This starts the server at http://127.0.0.1:8000
# Access it in your browser to test image upload and webcam feed
print('Starting FastAPI server...')
# Capture stdout and stderr to debug potential Uvicorn errors
fastapi_process = subprocess.Popen(
    ['uvicorn', 'main:app', '--host', '0.0.0.0', '--port', '8000'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True, # Decode stdout/stderr as text
    bufsize=1 # Line-buffered output
)

# Wait for the server to start
time.sleep(5) # Give it a few seconds to initialize

if fastapi_process.poll() is None:
    print('FastAPI server is running at http://127.0.0.1:8000')
    print('Open this URL in your browser to test the app.')
else:
    # If the process terminated, read its output for debugging
    stdout, stderr = fastapi_process.communicate()
    print("\n--- FastAPI Server STDOUT ---")
    print(stdout)
    print("\n--- FastAPI Server STDERR ---")
    print(stderr)
    raise RuntimeError(
        'Failed to start FastAPI server. Check the output above for Uvicorn errors.'
    )

# Keep the server running for 5 minutes to allow testing
# You can interrupt this cell (e.g., using the stop button in Jupyter) to stop the server manually.
try:
    print('\nFastAPI server will run for 5 minutes. Press the stop button to terminate early.')
    time.sleep(300)  # 5 minutes
finally:
    if fastapi_process.poll() is None: # Check if process is still running
        print('Terminating FastAPI server...')
        fastapi_process.terminate()
        fastapi_process.wait(timeout=10) # Wait a bit for it to terminate cleanly
        if fastapi_process.poll() is None:
            print("FastAPI server did not terminate, killing it...")
            fastapi_process.kill() # Force kill if it's still running
        print('FastAPI server stopped.')
    else:
        print('FastAPI server was already stopped or never started successfully.')


In [ ]:
# Optional: Convert best.pt to RKNN format
# Run this cell only after successfully testing the FastAPI app with best.pt
# Requires rknn-toolkit2 and a compatible environment
from pathlib import Path
import os # Import os for path checking

# Assume BASE_DIR is already defined from previous cells or define it here if running standalone
BASE_DIR = Path.cwd() # Adjust if your BASE_DIR is different
# Create convert_pt_to_rknn.py
convert_rknn_content = """from ultralytics import YOLO

# Load the YOLO model
model = YOLO('weights/best.pt')

# Export to RKNN format
model.export(format='rknn', name='rk3588')
"""

with open(BASE_DIR / 'convert_pt_to_rknn.py', 'w') as f:
    f.write(convert_rknn_content)

print('convert_pt_to_rknn.py created successfully.')

# Run conversion
# Note: This requires the RKNN toolkit and may need to be run on a compatible system
# Comment out the next line if you're not ready to convert
# %run convert_pt_to_rknn.py

# Move RKNN model to weights directory
# rknn_model_source = BASE_DIR / 'yolo11n_rknn_model' / 'best-rk3588.rknn'
# rknn_model_dest = BASE_DIR / 'weights' / 'best-rk3588.rknn'
# if rknn_model_source.exists():
#     shutil.move(rknn_model_source, rknn_model_dest)
#     print('RKNN model moved to weights directory.')
# else:
#     print('RKNN conversion did not produce the expected file. Check conversion logs.')

In [ ]:
# Optional: Create main1.py for RKNN deployment
# This is the FastAPI app modified for RKNN on OrangePi5Pro
# Run this cell to create the file, but execute it only on the OrangePi5Pro with the RKNN model

main1_py_content = """from fastapi import FastAPI, File, UploadFile, HTTPException
from fastapi.responses import HTMLResponse, StreamingResponse, JSONResponse
from fastapi.staticfiles import StaticFiles
from rknn.api import RKNN
import cv2
import numpy as np
from pathlib import Path
import settings
import io
from PIL import Image
import base64
import asyncio
import time

app = FastAPI()
app.mount('/static', StaticFiles(directory='static'), name='static')

# Update settings to use RKNN model
settings.DETECTION_MODEL = Path('weights/best-rk3588.rknn')

# Initialize RKNN model
model_path = Path(settings.DETECTION_MODEL)
try:
    rknn = RKNN(verbose=True)
    model_path_str = str(model_path)
    if rknn.load_rknn(model_path_str) != 0:
        raise Exception(f'Failed to load RKNN model from {model_path_str}')
    if rknn.init_runtime(target='rk3588') != 0:
        raise Exception('Failed to initialize RKNN runtime')
except Exception as ex:
    raise Exception(f'Unable to load RKNN model. Check the specified path: {model_path} - {ex}')

latest_webcam_results = {
    'recyclable': [],
    'non_recyclable': [],
    'hazardous': []
}

webcam_stop_event = asyncio.Event()

def classify_waste_type(detected_items):
    recyclable_items = set(detected_items) & set(settings.RECYCLABLE)
    non_recyclable_items = set(detected_items) & set(settings.NON_RECYCLABLE)
    hazardous_items = set(detected_items) & set(settings.HAZARDOUS)
    return recyclable_items, non_recyclable_items, hazardous_items

def remove_dash_from_class_name(class_name):
    return class_name.replace('_', ' ')

def preprocess_image_for_rknn(image_np):
    img_rgb = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)
    original_h, original_w = img_rgb.shape[:2]
    scale = min(settings.MODEL_INPUT_WIDTH / original_w, settings.MODEL_INPUT_HEIGHT / original_h)
    new_w, new_h = int(original_w * scale), int(original_h * scale)
    resized_img = cv2.resize(img_rgb, (new_w, new_h), interpolation=cv2.INTER_LINEAR)
    padded_img = np.full((settings.MODEL_INPUT_HEIGHT, settings.MODEL_INPUT_WIDTH, 3), 128, dtype=np.uint8)
    x_offset = (settings.MODEL_INPUT_WIDTH - new_w) // 2
    y_offset = (settings.MODEL_INPUT_HEIGHT - new_h) // 2
    padded_img[y_offset:y_offset+new_h, x_offset:x_offset+new_w] = resized_img
    input_data = np.expand_dims(padded_img, axis=0).astype(np.float32)
    print(f'DEBUG PREPROCESS - Original: {original_w}x{original_h}, Scaled: {new_w}x{new_h}, Padded: {padded_img.shape}')
    return input_data

def postprocess_yolov8_rknn_output(rknn_outputs, original_img_shape):
    print(f'DEBUG POSTPROCESS - RKNN outputs shapes: {[output.shape for output in rknn_outputs]}')
    predictions = rknn_outputs[0]
    predictions = predictions.transpose(0, 2, 1)
    predictions = predictions[0]
    print(f'DEBUG POSTPROCESS - Predictions shape after transpose: {predictions.shape}')

    num_classes = len(settings.ALL_CLASSES)
    img_h, img_w = original_img_shape
    input_h, input_w = settings.MODEL_INPUT_HEIGHT, settings.MODEL_INPUT_WIDTH

    boxes_raw = predictions[:, :4]
    scores = predictions[:, 4:]

    max_scores = np.max(scores, axis=1)
    class_ids = np.argmax(scores, axis=1)

    mask = max_scores >= settings.CONF_THRESHOLD
    boxes = boxes_raw[mask]
    max_scores = max_scores[mask]
    class_ids = class_ids[mask]

    if len(boxes) == 0:
        print('No detections after confidence filtering.')
        return [], [], [], original_img_shape

    scale = min(input_w / img_w, input_h / img_h)
    unpadded_w_in_model_coords = int(img_w * scale)
    unpadded_h_in_model_coords = int(img_h * scale)
    x_offset_in_model_coords = (input_w - unpadded_w_in_model_coords) // 2
    y_offset_in_model_coords = (input_h - unpadded_h_in_model_coords) // 2

    final_boxes_on_original = []
    for box in boxes:
        cx_padded, cy_padded, w_padded, h_padded = box
        cx_unpadded = cx_padded - x_offset_in_model_coords
        cy_unpadded = cy_padded - y_offset_in_model_coords
        cx_original = cx_unpadded / scale
        cy_original = cy_unpadded / scale
        w_original = w_padded / scale
        h_original = h_padded / scale
        x1_original = cx_original - (w_original / 2)
        y1_original = cy_original - (h_original / 2)
        x2_original = cx_original + (w_original / 2)
        y2_original = cy_original + (h_original / 2)
        x1_original = np.clip(x1_original, 0, img_w)
        y1_original = np.clip(y1_original, 0, img_h)
        x2_original = np.clip(x2_original, 0, img_w)
        y2_original = np.clip(y2_original, 0, img_h)
        final_boxes_on_original.append([int(x1_original), int(y1_original), int(x2_original), int(y2_original)])

    nms_boxes = [[x1, y1, x2-x1, y2-y1] for x1, y1, x2, y2 in final_boxes_on_original]
    indices = []
    if len(nms_boxes) > 0:
        indices = cv2.dnn.NMSBoxes(nms_boxes, max_scores.tolist(), settings.CONF_THRESHOLD, settings.NMS_IOU_THRESHOLD)

    final_boxes = []
    final_confidences = []
    final_class_ids = []

    if len(indices) > 0:
        indices = indices.flatten()
        final_boxes = [final_boxes_on_original[i] for i in indices]
        final_confidences = max_scores[indices]
        final_class_ids = class_ids[indices]

    detected_items = [settings.ALL_CLASSES[cls_id] for cls_id in final_class_ids if cls_id < num_classes]
    print(f'DEBUG POSTPROCESS - Detected items: {detected_items}')
    return final_boxes, final_confidences, final_class_ids, original_img_shape

def draw_boxes_on_image(image_np, boxes, confidences, class_ids, fps_text=''):
    image_np = image_np.copy()
    names = settings.ALL_CLASSES
    for i in range(len(boxes)):
        x1, y1, x2, y2 = boxes[i]
        confidence = confidences[i]
        class_id = class_ids[i]
        if class_id < len(names):
            class_name = names[class_id]
            color = (0, 255, 0)
            text = f'{remove_dash_from_class_name(class_name)}: {confidence:.2f}'
            cv2.rectangle(image_np, (x1, y1), (x2, y2), color, 2)
            cv2.putText(image_np, text, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    cv2.putText(image_np, fps_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, False)
    return image_np

@app.get('/', response_class=HTMLResponse)
async def serve_index():
    with open('static/index.html', 'r') as f:
        return f.read()

@app.post('/detect')
async def detect_image(file: UploadFile = File(...)):
    try:
        contents = await file.read()
        image = Image.open(io.BytesIO(contents)).convert('RGB')
        image_np = np.array(image)
        image_np = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)
        original_shape = image_np.shape[:2]
        input_image = preprocess_image_for_rknn(image_np)
        outputs = rknn.inference(inputs=[input_image])
        boxes, confidences, class_ids, _ = postprocess_yolov8_rknn_output(outputs, original_shape)
        detected_items = [settings.ALL_CLASSES[cls_id] for cls_id in class_ids if cls_id < len(settings.ALL_CLASSES)]
        recyclable_items, non_recyclable_items, hazardous_items = classify_waste_type(detected_items)
        result_dict = {
            'recyclable': [remove_dash_from_class_name(item) for item in recyclable_items],
            'non_recyclable': [remove_dash_from_class_name(item) for item in non_recyclable_items],
            'hazardous': [remove_dash_from_class_name(item) for item in hazardous_items]
        }
        plotted_image = draw_boxes_on_image(image_np, boxes, confidences, class_ids)
        _, buffer = cv2.imencode('.jpg', plotted_image)
        encoded_image = base64.b64encode(buffer).decode('utf-8')
        return {'results': result_dict, 'image': encoded_image}
    except Exception as e:
        print(f'Error during image detection: {e}')
        raise HTTPException(status_code=500, detail=str(e))

@app.get('/video_feed')
async def video_feed():
    global latest_webcam_results
    webcam_stop_event.clear()
    print('Backend: Webcam stream started, stop event cleared.')
    async def generate():
        cap = cv2.VideoCapture(settings.WEBCAM_PATH)
        if not cap.isOpened():
            print('Backend: Error: Could not open webcam.')
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' +
                   cv2.imencode('.jpg', np.zeros((480, 640, 3), dtype=np.uint8))[1].tobytes() + b'\r\n')
            return
        print('Backend: Webcam opened successfully. Streaming frames...')
        prev_frame_time = 0
        new_frame_time = 0
        try:
            while True:
                if webcam_stop_event.is_set():
                    print('Backend: Webcam stop event detected. Breaking loop.')
                    break
                success, frame = cap.read()
                if not success:
                    print('Backend: Error: Failed to read frame from webcam. Breaking loop.')
                    break
                new_frame_time = time.time()
                fps = 1 / (new_frame_time - prev_frame_time)
                prev_frame_time = new_frame_time
                fps_text = f'FPS: {int(fps)}'
                original_shape = frame.shape[:2]
                input_image = preprocess_image_for_rknn(frame)
                outputs = rknn.inference(inputs=[input_image])
                boxes, confidences, class_ids, _ = postprocess_yolov8_rknn_output(outputs, original_shape)
                detected_items = [settings.ALL_CLASSES[cls_id] for cls_id in class_ids if cls_id < len(settings.ALL_CLASSES)]
                recyclable_items, non_recyclable_items, hazardous_items = classify_waste_type(detected_items)
                latest_webcam_results['recyclable'] = [remove_dash_from_class_name(item) for item in recyclable_items]
                latest_webcam_results['non_recyclable'] = [remove_dash_from_class_name(item) for item in non_recyclable_items]
                latest_webcam_results['hazardous'] = [remove_dash_from_class_name(item) for item in hazardous_items]
                plotted_image = draw_boxes_on_image(frame, boxes, confidences, class_ids, fps_text)
                _, buffer = cv2.imencode('.jpg', plotted_image)
                frame_bytes = buffer.tobytes()
                yield (b'--frame\r\n'
                       b'Content-Type: image/jpeg\r\n\r\n' + frame_bytes + b'\r\n')
                await asyncio.sleep(0.001)
        finally:
            if cap.isOpened():
                cap.release()
                print('Backend: Webcam released.')
            else:
                print('Backend: Webcam was not opened, nothing to release.')
    return StreamingResponse(generate(), media_type='multipart/x-mixed-replace;boundary=frame')

@app.post('/stop_webcam_backend')
async def stop_webcam_backend():
    webcam_stop_event.set()
    print('Backend: Received stop signal from frontend. Event set.')
    return JSONResponse(content={'message': 'Webcam stop signal sent.'})

@app.get('/webcam_classification')
async def get_webcam_classification():
    return JSONResponse(content=latest_webcam_results)

@app.on_event('shutdown')
def cleanup():
    rknn.release()
    print('Backend: RKNN model released.')
"""

with open(BASE_DIR / 'main1.py', 'w') as f:
    f.write(main1_py_content)

print('main1.py created successfully.')

# Note: To run the RKNN FastAPI app, copy the following files to your OrangePi5Pro:
# - weights/best-rk3588.rknn
# - settings.py
# - main1.py
# - static/index.html
# Then run: uvicorn main1:app --host 0.0.0.0 --port 8000
# Ensure the RKNN toolkit and dependencies are installed on the OrangePi5Pro.